# 学习率

在上一个章节，我们尝试了使用梯度下降的方法进行模型训练。结果一顿操作猛如虎，为啥准确度还严重恶化了呢？

原因只有一个：步子迈得太大啦！

-----------------

梯度的本质是损失函数对模型参数的偏导数，是损失函数曲线在某一点的切线斜率。它是下降的方向，并不是下降的幅度。

梯度很大，只代表道路很陡峭，不代表步子应该很大。

步子太大，可能直接越过函数的最低点，造成损失值在最低点的左右震荡，无法收敛。最终导致模型训练失败。

-----------------

为此我们引进一个控制梯度的比例，称为**学习率**（Learning Rate）。这样梯度下降的公式变为：

$$
w_{new} = w_{old} - \eta \cdot \nabla J(w_{old})
$$

其中$\eta$为学习率。通过调整学习率，我们可以控制模型训练的步调。既不应太小，造成损失函数收敛太慢；也不能太大，造成损失函数无法收敛。

## 超参数

学习率是我们遇到的第一个**超参数**（Hyperparameter）。

我们已经知道模型参数包括权重和偏差。模型参数是模型训练的对象；而超参数则是我们控制模型训练步调的参数，比如学习率是我们控制梯度下降步调的参数。

超参数并不会随着模型训练得到优化，而是需要在模型训练开始前根据数据、经验等实际情况人为设定；并根据模型训练的效果进行调整，以期获得最佳训练结果。这个不断测试、调整超参数的过程，被称为**调参**（Tuning）。

In [13]:
import numpy as np

## 数据

### 特征、标签

In [14]:
feature = np.array([28.1, 58.0])
label = np.array([165])

## 模型

### 参数：权重、偏差

In [15]:
weight = np.ones([1, 2]) / 2
bias = np.zeros(1)

### 推理函数（前向传播）

In [16]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（平均平方差）

In [17]:
def mse_loss(p, y):
    return ((y - p) ** 2).mean()

### 梯度函数

In [18]:
def gradient(p, y):
    return - 2 * (y - p)

### 反向函数（反向传播）

改进后的反向函数利用学习率调整模型训练的幅度。

In [19]:
def backward(x, d, w, b, lr):
    w -= d * x * lr
    b -= np.sum(d) * lr
    return w, b

## 验证

### 超参数：学习率

学习率的设定是一个比较人为、主观的步骤，并且需要通过调参的过程来优化。通常的做法是从一个初始值（比如：0.1）开始，逐步缩小（比如：0.01、0.001）并尝试，以期找到合适的学习率。

在我们的例子中，通过观察在上一个章节中，没有学习率的情况下梯度下降的幅度，我们选择了一个相对较小的值。

In [20]:
LEARNING_RATE = 0.00001

### 推理

In [21]:
prediction = forward(feature, weight, bias)

print(f'prediction: {prediction}')

prediction: [43.05]


### 评估

In [22]:
error = mse_loss(prediction, label)

print(f'error: {error}')

error: 14871.802500000002


## 训练

### 反向传播

In [23]:
delta = gradient(prediction, label)
weight, bias = backward(feature, delta, weight, bias, LEARNING_RATE)

print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [[0.5685359 0.641462 ]]
bias: [0.002439]


可以看出，通过引入合适的学习率，经过一次模型训练，权重和偏差的调整幅度变得相对合理。实际结果如何呢？

### 重新评估

In [24]:
prediction = forward(feature, weight, bias)
error = mse_loss(prediction, label)

print(f'error: {error}')

error: 12503.020514375934


经过重新推理和评估，可以看出我们引入的学习率，经过一次模型训练，使损失值下降了大约20%。这已经在一个正常、合理的范围了。